# API gRPC Eigen Example using Python

In the following demo, we will be showing the basics of API gRPC protocol by means of a simple library we have created. This library basically contains two elements in its Python version:

* A **server** which basically implements an API gRPC interface to communicate with an installed library within it, in our case the Eigen library. This API gRPC interface basically exposes certain functionalities such as adding, subtracting and multiplying Eigen::VectorXd and Eigen::MatrixXd in a simple way.
* A **client** in charge of easing the interaction with the server by means of API gRPC interface specific methods. By using the client library, one does not need to know the specifics of the API gRPC interface. 

So, let's get started by importing our client:

In [ ]:
from ansys.eigen.python.grpc.client import DemoGRPCClient

This ``DemoGRPCClient`` class is basically the one which will handle the API gRPC interface with the server, together with the connection itself, the formatting of the request and so on. When constructing the class we must provide as inputs the ``ip`` and the ``port`` of the server. For this demo we are running, since we deployed both client and server in a docker-compose task, these containers will recognize each other by their docker container names, since they are in the same docker network. Thus, we will provide us arguments:

* IP(or DNS): python-grpc_server_1
* Port: 50051

The server is exposed by **IP 127.0.0.1** and **port 50051** as per defined in the Dockerfile of the server and the server itself. Thus, the previous inputs should be provided, although they are also the default values. Nonetheless, in the IP field we will provide the DNS for the sake of showing that DNS values are also accepted.

In [ ]:
my_client = DemoGRPCClient(ip="python-grpc_server_1", port=50051)

This ``DemoGRPCClient`` also has a method for verifying the connection to out client, which is a simple handshake/greeting, when we provide our name:

In [ ]:
print("=========================")
my_client.request_greeting("User")

This will let us verify that the connection to the server is adequate and communication is favorable.

Now, we will proceed to perform a simple operation like adding two 1D numpy.ndarrays. First, let us start by defining them:

In [ ]:
import numpy as np

vec_1 = np.array([1, 2, 3, 4], dtype=np.float64)
vec_2 = np.array([5, 4, 2, 0], dtype=np.float64)

Now, we will call the client method ``add_vectors(...)``, and we will explain the typical process of all interface methods (``add_XXXX(...)`` and ``multiply_XXXX(...)``):

* The client performs some sanity checks to confirm that the inputs provided are as expected. This Demo has some limitations such as: only 1D, 2D numpy.ndarrays are allowed; they must be of type float64. Direct interaction with the server (i.e. without a client) and using gRPC is out of the scope of this demo. Doing so could be considered as "impossible" since you would have to serialize your message on your own, following the standard defined in the proto files.

* The client serializes the messages with the inputs provided using generator functions. Our end server is characterized for receiving ``streams`` of messages, which basically represent a list of messages. Each of these messages are serialized following the interface proposed by the proto files, thanks to the automatically generated source code by protobuf.
    * For example our ``Vector`` message is characterized for having the following structure:
    
        ```protobuf
        enum DataType{
            INTEGER = 0;
            DOUBLE = 1;
        }
        ...
        message Vector {
            DataType data_type = 1;         // The kind of data inside the vector: INTEGER, DOUBLE values
            int32 vector_size = 2;          // The number of elements inside our vector
            bytes vector_as_chunk = 3;      // The vector itself as a chunk of bytes
        }
        ```

* When the server receives the messages, it deserializes them and interprets each of the previous fields. Thus, it is easily converted into a numpy.ndarray of the adequate type. Then, the desired vectors to be added are passed to the Eigen library via our ``demo_eigen_wrapper`` for the resolution of the demanded operation.

* Once the results of the operation are available, the server serializes the result and responds with the adequate message to the client.
    * For example, according to the proto file, our server receives a stream of Vector messages, and returns a single Vector message (which contains the result of the requested operation):
        ```protobuf
        
        // Adds two vectors
        rpc AddVectors(stream Vector) returns (Vector) {}
        ```

* The client then receives the response, deserializes the message and returns the corresponding result to the end-user as numpy.ndarray. Thus, the entire process is like a black-box for the end-user, and does not require to understand what is happening behind the scenes, since the end-user is only interested in the end-result.

Let us now call the method!

In [ ]:
# Call the client to perform your desired operation
vec_add = my_client.add_vectors(vec_1, vec_2)

# Show the result!
print("Vector addition!")
print("================")
print(vec_add)

As mentioned before, there are several other methods implemented:

In [ ]:
# Call the client to perform your desired operation
vec_flip = my_client.flip_vector(vec_1)

# Show the result!
print("Vector position-flip!")
print("====================")
print(vec_flip)

In [ ]:
# Call the client to perform your desired operation
vec_mul = my_client.multiply_vectors(vec_1, vec_2)

# Show the result!
print("Vector dot product!")
print("===================")
print(vec_mul)

Let us show as well operations with 2D numpy.ndarrays (i.e. Matrices)

In [ ]:
# Define your matrices
mat_1 = np.array([[1, 2], [3, 4]], dtype=np.float64)
mat_2 = np.array([[5, 4], [2, 0]], dtype=np.float64)

# Call the client to perform your desired operation
mat_add = my_client.add_matrices(mat_1, mat_2)

# Show the result!
print("Adding matrices!")
print("================")
print(mat_add)

In [ ]:
# Call the client to perform your desired operation
mat_mul = my_client.multiply_matrices(mat_1, mat_2)

# Show the result!
print("Multiplying matrices!")
print("=====================")
print(mat_mul)